# If Files present start from here

In [16]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [17]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(83000, 502) (21975, 502)


In [18]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [19]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10987])


In [20]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [21]:
# write to file.
filename = "LSTM_amazon.csv"
def write_to_csv(epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [22]:
import time
import numpy as np 
def train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device):
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    for epoch in range(epochs):
        start = time.time()
        model.train()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(loss.item())
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return model


def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    acc = correct/total
    append_to_csv(epochs, np.round(acc*100, 3))
    print("Accuracy:", (correct/total)*100)

In [23]:
import torch.optim as optim
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 10000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()
train(model, train_dl, val_dl, epochs, optimizer, loss_fn, device)

Epoch: 0, BCELoss: 0.36948525518717534
Val Loss: 0.301 	Val Acc: 0.872
Epoch: 1, BCELoss: 0.2753567851675921
Val Loss: 0.284 	Val Acc: 0.881
Epoch: 2, BCELoss: 0.24785845710570553
Val Loss: 0.275 	Val Acc: 0.886
Epoch: 3, BCELoss: 0.2286991954747453
Val Loss: 0.275 	Val Acc: 0.887
Epoch: 4, BCELoss: 0.20989364182284798
Val Loss: 0.288 	Val Acc: 0.886
Epoch: 5, BCELoss: 0.19103368558214012
Val Loss: 0.294 	Val Acc: 0.886
Epoch: 6, BCELoss: 0.17540876866733454
Val Loss: 0.313 	Val Acc: 0.886
Epoch: 7, BCELoss: 0.15179264582469162
Val Loss: 0.338 	Val Acc: 0.883
Epoch: 8, BCELoss: 0.13626214309696513
Val Loss: 0.365 	Val Acc: 0.881
Epoch: 9, BCELoss: 0.1192355654533414
Val Loss: 0.398 	Val Acc: 0.880


LSTMClassifier(
  (embedding): Embedding(10000, 32, padding_idx=0)
  (lstm): LSTM(32, 100)
  (dense): Linear(in_features=100, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [24]:
test(model, test_dl, epochs)

Accuracy: 88.20424137617184
